### Import Packages

In [ ]:
import json
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import PlaintextCorpusReader, stopwords

### Create Document Corpus

In [ ]:
corpus_dir = "./Literature-original"
corpus = PlaintextCorpusReader(corpus_dir, ".*\.txt")
file_names = corpus.fileids()
file_names

### Corpus documents preprocessing

In [ ]:
documents = {}
for file_name in file_names:
    documents[file_name] = corpus.raw(file_name)
print(json.dumps(documents, indent=4, ensure_ascii=False))

In [ ]:
lengths_pre = {}
for file_name in documents:
    lengths_pre[file_name] = len(word_tokenize(documents[file_name]))
print(json.dumps(lengths_pre, indent=4, ensure_ascii=False))

In [ ]:
ps = PorterStemmer()

In [ ]:
for file_name in documents:
    documents[file_name] = documents[file_name].lower()
    documents[file_name] = "".join([char for char in documents[file_name] if char not in string.punctuation])
    documents[file_name] = "".join([char for char in documents[file_name] if not char.isdigit()])
    documents[file_name] = " ".join([ps.stem(word) for word in word_tokenize(documents[file_name])])
    documents[file_name] = " ".join([word for word in word_tokenize(documents[file_name]) if word not in list(stopwords.words('english'))])
print(json.dumps(documents, indent=4, ensure_ascii=False))


In [ ]:
lengths_post = {}
for file_name in documents:
    lengths_post[file_name] = len(word_tokenize(documents[file_name]))
print(json.dumps(lengths_post, indent=4, ensure_ascii=False))